In [ ]:
!pip install -q huggingface_hub[hf_xet]
!pip install -q -U wandb

In [ ]:
import wandb
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HUGGINGFACE_TOKEN")

login(token = hf_token)

wb_token = user_secrets.get_secret("WANDB_API_KEY")
wandb.login(key=wb_token)

In [ ]:
from datasets import load_dataset

daa_file_path='/kaggle/input/building-the-fine-tuning-dataset/train.csv'

train_ds=load_dataset('csv', data_files=daa_file_path)['train']
train_ds

In [ ]:
train_ds[0]

# Pre-processing dataset

We will split dataset into training, testing and evaluation.

In [ ]:
train_test_split=train_ds.train_test_split(test_size=0.1, seed=42)
train_test_split

In [ ]:
train_eval_split=train_test_split['train'].train_test_split(test_size=0.1, seed=42)
train_eval_split

In [ ]:
from datasets import DatasetDict

findal_ds=DatasetDict({
    'train': train_eval_split['train'],
    'test': train_eval_split['test'],
    'eval': train_test_split['test']
})
findal_ds

In [ ]:
MODEL_NAME='all-MiniLM-L6-v2'
OUTPUT_DIR='encoder-L6-V2/training'
MODEL_DIR='user_name/encoder-L6-V2'

# Load the Model

In [ ]:
import torch
from sentence_transformers import SentenceTransformer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model=SentenceTransformer(MODEL_NAME).to(device)
model

In [10]:
from sentence_transformers.losses import MultipleNegativesRankingLoss

loss=MultipleNegativesRankingLoss(model)

In [ ]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, SimilarityFunction

eval_dataset = load_dataset("sentence-transformers/stsb", split="validation")

dev_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=eval_dataset["sentence1"],
    sentences2=eval_dataset["sentence2"],
    scores=eval_dataset["score"],
    main_similarity=SimilarityFunction.COSINE,
    name="sts-dev",
)

In [ ]:
from sentence_transformers import SentenceTransformerTrainingArguments
from sentence_transformers.training_args import BatchSamplers
from sentence_transformers import SentenceTransformerTrainer
from transformers import EarlyStoppingCallback


train_args=SentenceTransformerTrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=20,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=1e-4,
    warmup_ratio=0.1,
    fp16=True,
    bf16=False,
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # losses that use "in-batch negatives" benefit from no duplicates
    eval_strategy='steps',
    eval_steps=100,
    save_strategy='steps',
    save_steps=100,
    save_total_limit=2,
    logging_steps=100,
    report_to='wandb',
    run_name='encoder-L6-V2'
)

# evaluate the base model
dev_evaluator(model)

trainer=SentenceTransformerTrainer(
    model=model,
    args=train_args,
    train_dataset=findal_ds['train'],
    eval_dataset=findal_ds['test'],
    loss=loss,
    evaluator=dev_evaluator,
    
)

# early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=3)

# trainer.train(callbacks=[early_stopping_callback])

trainer.train()

In [ ]:
# evaluate the trained model on the test set
test_dataset = load_dataset("sentence-transformers/stsb", split="test")

test_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=eval_dataset["sentence1"],
    sentences2=eval_dataset["sentence2"],
    scores=eval_dataset["score"],
    main_similarity=SimilarityFunction.COSINE,
    name="sts-test",
)

test_evaluator(model)

In [14]:
model.save_pretrained(MODEL_DIR)

In [ ]:
import kagglehub

VARIATION_SLUG='v0.1.4'

handler = f'kaggle_user/encoder-L6-V2/transformers/{VARIATION_SLUG}'

kagglehub.model_upload(
    handle=handler,
    local_model_dir=MODEL_DIR,
    license_name='MIT',
    version_notes='FT with reasoning paths'
)